In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew, norm
from scipy.optimize import minimize
from numba import njit
from typing import Tuple, Optional
from enum import Enum
import qis
from dataclasses import fields, replace, asdict
from datetime import datetime, timedelta
import scipy.stats as ss
import copy
import scipy

from scipy.interpolate import splrep, BSpline
from numba.typed import List

# analytics
import sys
sys.path.insert(0,'../../') # just for jupyter  notebook
from stochvolmodels.pricers.hawkes_jd_pricer import HawkesJDParams, HawkesJDPricer, hawkesjd_chain_pricer, unpack_and_transform_pars_for_measure_change, unpack_pars

import tensorflow.experimental.numpy as tnp
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from stochvolmodels.data.test_option_chain import get_btc_test_chain_data, get_gld_test_chain_data_6m, get_sqqq_test_chain_data, get_spy_test_chain_data
from stochvolmodels.utils.funcs import to_flat_np_array, set_time_grid, timer, set_seed, transform_to_tfcomplex128, transform_from_tfcomplex128_to_np, slice_option_chain
from stochvolmodels.data.option_chain import OptionChain

import os
from stochvolmodels.pricers.core.bsm_pricer import infer_bsm_implied_vol, compute_bsm_price
import warnings
from stochvolmodels.MLE_estimator import hawkes_jd
from stochvolmodels.MLE_estimator import hawkes_jd_weekday

import pickle

warnings.filterwarnings('ignore')

In [37]:
def smooth_option_chain(option_chain, N_K=41):
    new_K_arr_ttms = []
    new_ivs_ttms = []
    new_optiontypes_ttms = []

    for i in range(len(option_chain.strikes_ttms)):
        K_arr = option_chain.strikes_ttms[i]
        mid_ivs = (option_chain.bid_ivs[i]+option_chain.ask_ivs[i])/2
        bid_ask_spread = np.max(option_chain.bid_ivs[i]+option_chain.ask_ivs[i])
        
        min_K = option_chain.strikes_ttms[i].min()
        max_K = option_chain.strikes_ttms[i].max()
        new_K_arr = np.linspace(min_K, max_K, N_K)
        try:
            tck = splrep(K_arr, mid_ivs, s=bid_ask_spread)
            new_ivs = BSpline(*tck)(new_K_arr)
        except:
            tck = splrep(K_arr, mid_ivs, k=2)
            new_ivs = BSpline(*tck)(new_K_arr)
        new_optiontypes = np.array(['P']*len(new_K_arr), dtype='<U1')
        C_id = new_K_arr < option_chain.forwards[i]
        new_optiontypes[C_id] = 'C'
        
        new_K_arr_ttms.append(new_K_arr)
        new_ivs_ttms.append(new_ivs)
        
        new_optiontypes_ttms.append(new_optiontypes)
        
    new_option_chain = OptionChain(ids=option_chain.ids,
        ttms=option_chain.ttms,
        ticker='BTC',
        forwards=option_chain.forwards,
        strikes_ttms=new_K_arr_ttms,
        optiontypes_ttms=new_optiontypes_ttms,
        discfactors=option_chain.discfactors,
        bid_ivs=new_ivs_ttms,
        ask_ivs=new_ivs_ttms)

    return new_option_chain
    


In [38]:
def slice_option_chain(option_chain, ids_i):
    option_chain = copy.copy(option_chain)
    replacements = dict()
    for field in fields(option_chain):
        field_name = field.name
        option_chain_value = getattr(option_chain, field_name)
        try:
            replacements[field_name] = [option_chain_value[i] for i in ids_i]
        except:
            replacements[field_name] = option_chain_value
            
    return OptionChain(**replacements)

In [39]:
with open('BTC_option_chain.pickle', 'rb') as f:
    BTC_option_chain_dict = pickle.load(f)
    
P_params = pd.read_csv('P_params.csv', index_col=0)

In [40]:
BTC_option_chain_dict['2020-03-13'].bid_ivs

[array([1.41149994]),
 array([1.30759995]),
 array([1.10440002]),
 array([1.04709999]),
 array([1.00269997])]

In [41]:
dates_arr = pd.Series(BTC_option_chain_dict.keys())

year = 'refine'

# dates_arr = list(dates_arr.loc[_id])

In [42]:
theta_p = P_params.loc[:,'theta_p'].iloc[0]
theta_m = P_params.loc[:,'theta_m'].iloc[0]
kappa_p = P_params.loc[:,'kappa_p'].iloc[0]
kappa_m = P_params.loc[:,'kappa_m'].iloc[0]
beta11  = P_params.loc[:,'beta11'].iloc[0]
beta12  = P_params.loc[:,'beta12'].iloc[0]
beta21  = P_params.loc[:,'beta21'].iloc[0]
beta22  = P_params.loc[:,'beta22'].iloc[0]
eta_p   = P_params.loc[:,'eta_p'].iloc[0]
eta_m   = P_params.loc[:,'eta_m'].iloc[0]
nu_p    = P_params.loc[:,'nu_p'].iloc[0]
nu_m    = P_params.loc[:,'nu_m'].iloc[0]

mu = 0
sigma = P_params.sigma.iloc[0]


In [43]:
def smooth_option_chain(option_chain, N_K=41):
    new_K_arr_ttms = []
    new_ivs_ttms = []
    new_optiontypes_ttms = []

    for i in range(len(option_chain.strikes_ttms)):
        K_arr = option_chain.strikes_ttms[i]
        mid_ivs = (option_chain.bid_ivs[i]+option_chain.ask_ivs[i])/2
        bid_ask_spread = np.max(option_chain.bid_ivs[i]+option_chain.ask_ivs[i])
        
        min_K = option_chain.strikes_ttms[i].min()
        max_K = option_chain.strikes_ttms[i].max()
        new_K_arr = np.linspace(min_K, max_K, N_K)

        tck = splrep(K_arr, mid_ivs, s=bid_ask_spread)
        new_ivs = BSpline(*tck)(new_K_arr)
        new_optiontypes = np.array(['P']*len(new_K_arr), dtype='<U1')
        C_id = new_K_arr < option_chain.forwards[i]
        new_optiontypes[C_id] = 'C'
        
        new_K_arr_ttms.append(new_K_arr)
        new_ivs_ttms.append(new_ivs)
        
        new_optiontypes_ttms.append(new_optiontypes)
        
    new_option_chain = OptionChain(ids=option_chain.ids,
        ttms=option_chain.ttms,
        ticker='BTC',
        forwards=option_chain.forwards,
        strikes_ttms=new_K_arr_ttms,
        optiontypes_ttms=new_optiontypes_ttms,
        discfactors=option_chain.discfactors,
        bid_ivs=new_ivs_ttms,
        ask_ivs=new_ivs_ttms)

    return new_option_chain
    
    
# plt.scatter(K_arr, option_chain.bid_ivs[i], color='green')
# plt.scatter(K_arr, option_chain.ask_ivs[i], color='red')

# plt.scatter(K_arr, mid_ivs)
# plt.scatter(new_K_arr, new_ivs)
    

In [44]:
Q_results_1m = dict()

with open('Q_results2019.pickle', 'rb') as f:
    Q_results_2019 = pickle.load(f)
    
with open('Q_results2020.pickle', 'rb') as f:
    Q_results_2020 = pickle.load(f)
    
with open('Q_results2021.pickle', 'rb') as f:
    Q_results_2021 = pickle.load(f)
    
with open('Q_results2022.pickle', 'rb') as f:
    Q_results_2022 = pickle.load(f)
    
with open('Q_results2023.pickle', 'rb') as f:
    Q_results_2023 = pickle.load(f)
    
Q_results_1m.update(Q_results_2019)
Q_results_1m.update(Q_results_2020)    
Q_results_1m.update(Q_results_2021)
Q_results_1m.update(Q_results_2022)
Q_results_1m.update(Q_results_2023)

In [24]:
# with open('Q_results_refined.pickle', 'rb') as f:
#     Q_results_1m = pickle.load(f)

In [25]:
pricer = HawkesJDPricer(M=2**10, x_max=8, n_steps_per_ttm=100)

bounds = ( (0.12, 2), (None, None), (None, None) )
method = 'powell'
options = {'ftol':1e-8, 'tol':1e-8, 'maxiter':1e8}
errors_weights = (1, 0, 0, 0, 0)
chi = 0
# chi0_arr = [(0, 0), (chi, chi), (-chi, -chi), (-chi, chi), (-chi, chi)]
chi0_arr = [(-chi,-chi)]
mu = 0
sigma = .45

MAPE_thres = 0.5
# for date in dates_arr:
for date in ['2022-06-18']:

    print('-----------', date, '-----------')
    if (date in Q_results_1m):
        if (Q_results_1m[date]['MAPE'] > MAPE_thres) or (np.isnan(Q_results_1m[date]['MAPE'])) or (Q_results_1m[date]['measure_change_params'][1]< -15):
            pass
        else:
            continue
    else:
        pass
    lambda_p = P_params.loc[date,:].lambda_p_right
    lambda_m = P_params.loc[date,:].lambda_m_right
    is_pos_jump = P_params.loc[date,:].jump_sizes > 0 
    is_neg_jump = P_params.loc[date,:].jump_sizes < 0 
    
    # if (date in Q_results_1m):
    #     if (Q_results_1m[date]['MAPE'] > MAPE_thres) or (np.isnan(Q_results_1m[date]['MAPE'])):
    #         sigma = .1
    #         # sigma = MLE_estimator_daily.sigma
    #         xi_p0 = 0
    #         xi_m0 = 0
    # else:
    #     pass 

    params = HawkesJDParams(mu=mu, sigma=sigma,
                            eta_p=eta_p, nu_p=nu_p, eta_m=eta_m, nu_m=nu_m,
                            theta_p=theta_p, kappa_p=kappa_p, theta_m=theta_m, kappa_m=kappa_m,
                            beta11=beta11, beta21=beta21, beta12=beta12, beta22=beta22,
                            lambda_p=lambda_p, lambda_m=lambda_m)
    
    params = transform_to_tfcomplex128(params)
    
    targeted_ids = ['3w']
    option_ids_i = [i for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
    option_ids   = [_id for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
    option_slice = slice_option_chain(BTC_option_chain_dict[date], option_ids_i)

    if len(option_ids_i) == 0:
        targeted_ids = ['2w']
        option_ids_i = [i for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
        option_ids   = [_id for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
        option_slice = slice_option_chain(BTC_option_chain_dict[date], option_ids_i)
        
    if len(option_ids_i) == 0:
        targeted_ids = ['1m']
        option_ids_i = [i for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
        option_ids   = [_id for i, _id in enumerate(BTC_option_chain_dict[date].ids) if _id in targeted_ids]
        option_slice = slice_option_chain(BTC_option_chain_dict[date], option_ids_i)

    new_option_chain = smooth_option_chain(option_slice)

    results = []
    MAPEs = []
    for chi0 in chi0_arr:
        chi_p0, chi_m0 = chi0
        measure_change_params, measure_change_results = pricer.calibrate_measure_change_params_to_chain(option_chain=new_option_chain,
                                                                                                        params0 = params,
                                                                                                        bounds  = bounds,
                                                                                                        method  = method,
                                                                                                        errors_weights = errors_weights, 
                                                                                                        options = options,
                                                                                                        is_vega_weighted = True, 
                                                                                                        p0      = (sigma, chi_p0, chi_m0),
                                                                                                        verbose = True)
        
        _bid_ivs = np.concatenate(option_slice.bid_ivs)
        _ask_ivs = np.concatenate(option_slice.ask_ivs)
        _mid_ivs = (_bid_ivs + _ask_ivs) / 2

        model_ivs = pricer.compute_model_ivols_for_chain(option_chain=option_slice, params=measure_change_results)
        _model_ivs = np.concatenate(model_ivs)
        MAPE = np.mean( np.abs(_model_ivs - _mid_ivs)/_mid_ivs )
        results.append((measure_change_params, measure_change_results))
        MAPEs.append(MAPE)
        
        
    measure_change_params, measure_change_results = results[np.argmin(MAPEs)]
    MAPE = np.min(MAPEs)
    
    result = {'measure_change_params':measure_change_params, 'measure_change_results': measure_change_results,
              'MAPE':MAPE, 'n_options':len(_mid_ivs), 'is_pos_jump':is_pos_jump, 'is_neg_jump':is_neg_jump, 'option_ids':option_ids}
    print(date, measure_change_params, MAPE, len(_mid_ivs), is_pos_jump, is_neg_jump, option_ids)
    
    if MAPE < Q_results_1m[date]['MAPE']+.1:
        Q_results_1m[date] = result
        print('Updated.')

In [51]:
with open('Q_results_refined.pickle', 'wb') as f:
    pickle.dump(Q_results_1m, f)